In [1]:
import time, math, os, importlib,sys
import sklearn.metrics  
import arcgisscripting
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio as rio
import rasterio.mask
import rasterio.plot as rio_pl
import matplotlib.image as mpimg
from datetime import datetime

from rasterio.plot import show
from rasterio.transform import Affine
from rasterio.mask import mask
from rasterio import MemoryFile
from rasterio.profiles import DefaultGTiffProfile
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.stats import sem
from sklearn.metrics import mean_squared_error
from shapely.geometry import box, Polygon, Point
from shapely import wkt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import contextily as cx

import arcpy
from arcpy.sa import *
import misc.preprocess
import misc.interpolation
arcpy.env.overwriteOutput = True

# 1. Load and Preprocess Data <a class="anchor" id="load_data"></a>

In [2]:
# path = "E:/Projects/SEACAR_WQ_Pilot/"
# gis_path = path+'GIS_data/'

path = "D:/Water_Quailty/"
gis_path = path+'GIS_data/'

In [3]:
# Reload modules in external .py files after editing.
dfAll = pd.read_csv(path + "all_0214.csv").drop(columns=['Unnamed: 0','Unnamed: 0.1','RowID','ValueQualifier'])
# Convert SampleDate froms str to date
dfAll['SampleDate'] = pd.to_datetime(dfAll['SampleDate']).dt.date

C:\Users\xiang11\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
col_ls = ['RowID','ParameterName','ParameterUnits','ProgramLocationID','ActivityType','ManagedAreaName',
                   'SampleDate','Year','Month','ResultValue','ValueQualifier','Latitude_DD','Longitude_DD']
para_ls = ["Salinity","Total Nitrogen","Dissolved Oxygen","Turbidity","Secchi Depth"]
para_ls_ab = ["S","TN","DO","T","SD"]
# Convert full MA names to short names
dictArea    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'Charlotte Harbor','Big Bend Seagrasses Aquatic Preserve':'Big Bend',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'GTM Reserve','Estero Bay Aquatic Preserve':'Estero Bay',
                'Biscayne Bay Aquatic Preserve':'Biscayne Bay','Matlacha Pass Aquatic Preserve':'Matlacha Pass AP',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island'}

# Convert full MA names to MA name in ORCP_Managed_Areas_Oct2021
dictArea2    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'Gasparilla Sound-Charlotte Harbor','Big Bend Seagrasses Aquatic Preserve':'Big Bend Seagrasses',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'Guana Tolomato Matanzas NERR','Estero Bay Aquatic Preserve':'Estero Bay',
                'Biscayne Bay Aquatic Preserve':'Biscayne Bay','Matlacha Pass Aquatic Preserve':'Matlacha Pass',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island Sound'}
dictArea3    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'ch','Big Bend Seagrasses Aquatic Preserve':'bb',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'gtm','Estero Bay Aquatic Preserve':'eb',
                'Biscayne Bay Aquatic Preserve':'bbay','Matlacha Pass Aquatic Preserve':'Matlacha Pass AP',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island'}

dictPara = {"Salinity":'S','Total Nitrogen':'TN','Dissolved Oxygen':'DO','Turbidity':'T','Secchi Depth':'SD'}
dictUnits   = {"Salinity":"ppt","Total Nitrogen": "mg/L","Dissolved Oxygen": "mg/L","Turbidity": "NTU", "Secchi Depth": "m"}
listArea    = dfAll["ManagedAreaName"].unique()
listPara    = ["Salinity","Total Nitrogen","Dissolved Oxygen","Turbidity","Secchi Depth"]
SpatialRef = '3086'

# 2. Combine Discrete and Continuous Data <a class="anchor" id="combine"></a>

Combine dis and con dataframes

In [5]:
def interpolation_auto(method,dataframe,managed_area,Year,Season,start_date,end_date,parameter,covariates,out_raster,out_ga_layer,predict_std_err):
    method = method
    dataframe = dataframe
    Area   = managed_area
    Year   = Year
    Season = Season
    start_date,end_date = start_date,end_date
    Para   = parameter
    covariates = covariates
    fname = [dictArea[Area],Year,Season[0:3],dictPara[Para]]
    
    input_pt = gis_path+"input_point/{}/{}{}_{}.shp".format(*fname)
    
    df,gdf= misc.preprocess.select_aggr_area_season(dataframe,start_date,end_date, Area, Para)
    
    try:
        gdf   = gdf.to_crs(int(SpatialRef))
        boundary_shp = gis_path+ 'managed_area_boundary/{}.shp'.format(dictArea[Area][0:3])
        gdf.to_file(input_pt,driver='ESRI Shapefile',crs="EPSG:"+SpatialRef)
        MA = gpd.read_file(gis_path + r"managed_area_boundary/ORCP_Managed_Areas_Oct2021.shp")
        boundary = MA[MA['MA_Name']==dictArea2[Area]].to_crs(int(SpatialRef))
        boundary.to_file(boundary_shp , driver='ESRI Shapefile',crs="EPSG:"+SpatialRef)
        extent = str(boundary.geometry.total_bounds).replace('[','').replace(']','')

        if type(covariates) == str:
            in_explanatory_rasters = gis_path + "covariates/{}/{}.tif".format(covariates, dictArea[Area])
        elif type(covariates) == list:
            in_explanatory_rasters = []
            for i in range(len(covariates)):
                in_explanatory_raster = str(gis_path + "covariates/{}/{}.tif".format(covariates[i], dictArea[Area]))
                in_explanatory_rasters.append(in_explanatory_raster)

        in_features = input_pt
        out_raster = gis_path +"output_raster/{}/{}{}_{}.tif".format(*out_raster)
        value_field = "ResultValu"
        out_ga_layer = gis_path +"ga_layer/{}/{}{}_{}.lyrx".format(*out_ga_layer)
        ga_to_raster = gis_path + 'standard_error_prediction/{}/{}{}_{}.tif'.format(*predict_std_err)
        in_explanatory_rasters = in_explanatory_rasters
        mask = gis_path+ '{}.shp'.format(dictArea3[Area])

        try:
            Result,Stat = misc.interpolation.interpolation(
                            method = method, input_point = in_features, out_raster = out_raster, 
                            z_field = value_field, out_ga_layer = out_ga_layer, extent = extent, 
                            mask = mask, ga_to_raster = ga_to_raster, in_explanatory_rasters = in_explanatory_rasters)
            return out_raster,out_ga_layer,ga_to_raster
        except Exception:
            e = sys.exc_info()[1]
            print(Para + " in " + str(Year) + " " + Season + " caused an error:")
            print(e.args[0])
            return np.nan,np.nan,np.nan    
    except Exception:
            e = sys.exc_info()[1]
            print(Para + " in " + str(Year) + " " + Season + " caused an error:")
            print(e.args[0])
            return np.nan,np.nan,np.nan            

In [10]:
dfSeason = pd.read_csv(path + "OEATUSF_Geospatial_TempSeasons_update_0502.csv")
dfSeason = dfSeason.dropna(subset=["s_start","s_end"]).reset_index(drop=True)

dfSeason['start_date'] = pd.to_datetime(dfSeason['s_start'])
dfSeason["end_date"] = pd.to_datetime(dfSeason["s_end"])
dfSeason["date_range"] = dfSeason["end_date"] - dfSeason["start_date"]
dfSeason["daterange"] = dfSeason.apply(lambda x: int(str(x["date_range"]).split(" ")[0]),axis=1)
dfSeason = dfSeason[dfSeason["daterange"]>0].drop(columns="daterange").reset_index(drop=True)
dfSeason.head()

,Unnamed: 0,param,ma,st_Year,season,med_seamo_ma,s_start,s_end,covariates,start_date,end_date,date_range
0,147,temp,Estero Bay Aquatic Preserve,2022,Summer,7-Jun,6/7/2022,8/26/2022,bathymetry+LDI+popden,2022-06-07,2022-08-26,80 days
1,71,temp,Biscayne Bay Aquatic Preserve,2022,Summer,1-Jun,6/1/2022,8/26/2022,bathymetry+LDI+popden,2022-06-01,2022-08-26,86 days
2,70,temp,Biscayne Bay Aquatic Preserve,2022,Spring,2-Mar,3/2/2022,5/31/2022,bathymetry+LDI+popden,2022-03-02,2022-05-31,90 days
3,146,temp,Estero Bay Aquatic Preserve,2022,Spring,27-Feb,2/27/2022,6/6/2022,bathymetry+LDI+popden,2022-02-27,2022-06-06,99 days
4,69,temp,Biscayne Bay Aquatic Preserve,2022,Winter,8-Feb,2/8/2022,3/1/2022,bathymetry+LDI+popden,2022-02-08,2022-03-01,21 days


In [11]:
# Print all managed areas
display(dfSeason['ma'].unique())

# Select the first 3 managed areas
data = dfSeason.assign(**{'Total Nitrogen': np.nan, 'Dissolved Oxygen': np.nan, 'Salinity': np.nan, 'Secchi Depth': np.nan, 'Turbidity': np.nan})

array(['Estero Bay Aquatic Preserve', 'Biscayne Bay Aquatic Preserve',
       'Guana Tolomato Matanzas National Estuarine Research Reserve',
       'Gasparilla Sound-Charlotte Harbor Aquatic Preserve',
       'Big Bend Seagrasses Aquatic Preserve'], dtype=object)

In [63]:
listPara[3:5]

['Turbidity', 'Secchi Depth']

In [92]:
a = dfSeason[(dfSeason['ma'] == 'Estero Bay Aquatic Preserve') & (dfSeason["st_Year"] == 2022)]
a

,Unnamed: 0,param,ma,st_Year,season,med_seamo_ma,s_start,s_end,covariates
0,145,temp,Estero Bay Aquatic Preserve,2022,Winter,25-Dec,12/25/2022,2/26/2022,bathymetry+LDI+popden
1,147,temp,Estero Bay Aquatic Preserve,2022,Summer,7-Jun,6/7/2022,8/26/2022,bathymetry+LDI+popden
4,146,temp,Estero Bay Aquatic Preserve,2022,Spring,27-Feb,2/27/2022,6/6/2022,bathymetry+LDI+popden


In [106]:
dfAll[(dfAll['ManagedAreaName'] == 'Biscayne Bay Aquatic Preserve') & (dfAll["Year"] == 2022) & ((dfAll["ParameterName"] == 'Secchi Depth'))]

,ParameterName,ParameterUnits,ProgramLocationID,ActivityType,ManagedAreaName,SampleDate,Year,Month,ResultValue,Latitude_DD,Longitude_DD,timestamp


Run through the seasonality table

In [104]:
for i in data.index:
    for para in listPara[3:5]:
        name = [dictArea[data.iloc[i]["ma"]],data.iloc[i]["st_Year"],data.iloc[i]["season"],dictPara[para]]
        print(name)
#         out_raster,out_ga_layer,ga_to_raster = interpolation_auto(method = "rk",
#                            dataframe = dfAll,
#                            managed_area = data.iloc[i]["ma"],
#                            Year = data.iloc[i]["st_Year"],
#                            Season = data.iloc[i]["season"],
#                            start_date = data.iloc[i]["s_start"],
#                            end_date = data.iloc[i]["s_end"],
#                            parameter = para,
#                            covariates = data.iloc[i]["covariates"].split("+"),
#                            out_raster = name,
#                            out_ga_layer = name,
#                            predict_std_err = name)
#         #data.iloc[i]['raster'],data.iloc[i]['GA_layer'],data.iloc[i]['StErrPred'] = out_raster,out_ga_layer,ga_to_raster
#         data.loc[i, para] = out_raster
#         data.to_csv(path + 'output.csv')
#         display('Interpolated row:{}, {}'.format(i,name))

['Estero Bay', 2022, 'Summer', 'T']
['Estero Bay', 2022, 'Summer', 'SD']
['Biscayne Bay', 2022, 'Summer', 'T']
['Biscayne Bay', 2022, 'Summer', 'SD']
['Biscayne Bay', 2022, 'Spring', 'T']
['Biscayne Bay', 2022, 'Spring', 'SD']
['Estero Bay', 2022, 'Spring', 'T']
['Estero Bay', 2022, 'Spring', 'SD']
['Biscayne Bay', 2022, 'Winter', 'T']
['Biscayne Bay', 2022, 'Winter', 'SD']
['Biscayne Bay', 2021, 'Fall', 'T']
['Biscayne Bay', 2021, 'Fall', 'SD']
['Estero Bay', 2021, 'Fall', 'T']
['Estero Bay', 2021, 'Fall', 'SD']
['GTM Reserve', 2021, 'Summer', 'T']
['GTM Reserve', 2021, 'Summer', 'SD']
['Estero Bay', 2021, 'Summer', 'T']
['Estero Bay', 2021, 'Summer', 'SD']
['Matlacha Pass AP', 2021, 'Summer', 'T']
['Matlacha Pass AP', 2021, 'Summer', 'SD']
['Biscayne Bay', 2021, 'Summer', 'T']
['Biscayne Bay', 2021, 'Summer', 'SD']
['Biscayne Bay', 2021, 'Spring', 'T']
['Biscayne Bay', 2021, 'Spring', 'SD']
['Matlacha Pass AP', 2021, 'Winter', 'T']
['Matlacha Pass AP', 2021, 'Winter', 'SD']
['Estero 

['Big Bend', 2009, 'Fall', 'T']
['Big Bend', 2009, 'Fall', 'SD']
['Estero Bay', 2009, 'Fall', 'T']
['Estero Bay', 2009, 'Fall', 'SD']
['GTM Reserve', 2009, 'Fall', 'T']
['GTM Reserve', 2009, 'Fall', 'SD']
['Matlacha Pass AP', 2009, 'Fall', 'T']
['Matlacha Pass AP', 2009, 'Fall', 'SD']
['GTM Reserve', 2009, 'Summer', 'T']
['GTM Reserve', 2009, 'Summer', 'SD']
['Big Bend', 2009, 'Summer', 'T']
['Big Bend', 2009, 'Summer', 'SD']
['Estero Bay', 2009, 'Summer', 'T']
['Estero Bay', 2009, 'Summer', 'SD']
['Matlacha Pass AP', 2009, 'Summer', 'T']
['Matlacha Pass AP', 2009, 'Summer', 'SD']
['Matlacha Pass AP', 2009, 'Spring', 'T']
['Matlacha Pass AP', 2009, 'Spring', 'SD']
['Estero Bay', 2009, 'Spring', 'T']
['Estero Bay', 2009, 'Spring', 'SD']
['GTM Reserve', 2009, 'Spring', 'T']
['GTM Reserve', 2009, 'Spring', 'SD']
['Big Bend', 2009, 'Spring', 'T']
['Big Bend', 2009, 'Spring', 'SD']
['GTM Reserve', 2009, 'Winter', 'T']
['GTM Reserve', 2009, 'Winter', 'SD']
['Big Bend', 2009, 'Winter', 'T']
[

In [ ]:
for i in data.index:
    for para in listPara[3:5]:
        name = [dictArea[data.iloc[i]["ma"]],data.iloc[i]["st_Year"],data.iloc[i]["season"],dictPara[para]]
        out_raster,out_ga_layer,ga_to_raster = interpolation_auto(method = "rk",
                           dataframe = dfAll,
                           managed_area = data.iloc[i]["ma"],
                           Year = data.iloc[i]["st_Year"],
                           Season = data.iloc[i]["season"],
                           start_date = data.iloc[i]["s_start"],
                           end_date = data.iloc[i]["s_end"],
                           parameter = para,
                           covariates = data.iloc[i]["covariates"].split("+"),
                           out_raster = name,
                           out_ga_layer = name,
                           predict_std_err = name)
        #data.iloc[i]['raster'],data.iloc[i]['GA_layer'],data.iloc[i]['StErrPred'] = out_raster,out_ga_layer,ga_to_raster
        data.loc[i, para] = out_raster
        data.to_csv(path + 'output.csv')
        display('Interpolated row:{}, {}'.format(i,name))

C:\Users\xiang11\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


Start the interpolation with the RK method
--- Time lapse: 59.78075098991394 seconds ---


"Interpolated row:0, ['Estero Bay', 2022, 'Summer', 'T']"

Start the interpolation with the RK method
Secchi Depth in 2022 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).



"Interpolated row:0, ['Estero Bay', 2022, 'Summer', 'SD']"

...

Start the interpolation with the RK method
--- Time lapse: 69.75648975372314 seconds ---


"Interpolated row:3, ['Estero Bay', 2022, 'Spring', 'T']"

Start the interpolation with the RK method
Secchi Depth in 2022 Spring caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).



"Interpolated row:3, ['Estero Bay', 2022, 'Spring', 'SD']"

Start the interpolation with the RK method
